In [3]:
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import re
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import folium
from folium.plugins import MarkerCluster
from branca.element import Template, MacroElement


In [36]:
r=pd.read_pickle('indeed_job_co.pkl')

r.head()

,Title,Company,Location,Salary,URL,job_summary
0,"Data ScientistTikTok3.5Mountain View, CA+27 lo...",TikTok,"Mountain View, CA+27 locations",not_listed,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
1,newClinical Studies Data ScientistTailored Man...,Tailored Management,Remote,$54.34 an hour,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
2,newData Scientist or BioinformaticianUT Southw...,UT Southwestern Medical Center,+4 locationsRemote,"$60,000 - $100,000 a year",http://www.indeed.com/company/UT-Southwestern-...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
3,Data Scientist (Entry Level- Remote)HiLabsRale...,HiLabs,"Raleigh-Durham, NC+1 location•Remote","$120,000 - $150,000 a year",http://www.indeed.com/company/HILABS/jobs/Data...,"<div class=""jobsearch-jobDescriptionText"" id=""..."
4,newData Scientist - Full RemoteVaco3.7+1 locat...,Vaco,+1 locationRemote,"$145,000 - $170,000 a year",http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""..."


# Data Cleaning
* Salary

In [38]:
# converting Salary into string
r.Salary=r.Salary.astype(str)


# removing values in salary column which are not numbers and replacing it with NAN
def salary(x):
    if 'not_listed' in x:
        return np.nan
    
    elif 'Full_time' in x:
        return np.nan
    
    elif 'Full-time' in x:
        return np.nan
    
    elif 'Full_time ' in x:
        return np.nan
    
    elif 'Part-time +1' in x:
        return np.nan
    
    elif 'Part-time +2' in x:
        return np.nan
    
    elif 'Full-time +1' in x:
        return np.nan
    
    elif 'Temporary +3' in x:
        return np.nan
    
    else:
        return x
    
# apply the above function to all the rows of Salary column
r['Salary']=r['Salary'].apply(lambda x: salary(x))

# cleaning Salary column to have digits only
r.Salary=r.Salary.astype(str)
r['Salary']=r['Salary'].apply(lambda x: x.strip('a year') if 'a year' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip('$') if '$' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip(' $') if ' $' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip('a year') if 'a year' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip(' a year') if ' a year' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip('an hour') if 'an hour' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip('a month') if 'a month' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip('From') if 'From' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.strip('Up to') if 'Up to' in x else x)
r['Salary']=r['Salary'].apply(lambda x: x.replace(',','') if ',' in x else x)
r['Salary']=r['Salary'].apply(lambda x: np.NaN if 'Contract' in x else x)
r.Salary=r.Salary.astype(str)
r['Salary']=r['Salary'].apply(lambda x: np.NaN if 'Internship' in x else x)
r.Salary=r.Salary.astype(str)
r['Salary']=r['Salary'].apply(lambda x: np.NaN if 'Temporary +1' in x else x)

# creating min and max salary and splitting the values between columns
r.Salary=r.Salary.astype(str)
r['min_salary']=r['Salary'].apply(lambda x: x.split('-')[0] if '-' in x else x)
r.Salary=r.Salary.astype(str)
r['max_salary']=r['Salary'].apply(lambda x: x.split('-')[1] if '-' in x else x)
r['max_salary']=r['max_salary'].apply(lambda x: x.strip('$') if '$' in x else x)
r['max_salary']=r['max_salary'].apply(lambda x: x.strip(' $') if ' $' in x else x)
r['min_salary']=r['min_salary'].apply(lambda x: x.strip(' ') if ' ' in x else x)
r['max_salary']=r['max_salary'].apply(lambda x: x.split(' ') if ' ' in x else x)

# converting min and max to floats
r.min_salary=r.min_salary.astype(float)
r.max_salary=r.max_salary.astype(float)


# Converting hourly salaries into yearly by multiplying it by 2080(40 hours per week * 52 weeks in year )
r['min_salary']=r['min_salary'].apply(lambda x: x*2080 if x < 300 else x)
r['max_salary']=r['max_salary'].apply(lambda x: x*2080 if x < 300 else x)

# Converting monthly salaries into yearly by multiplying by it by 12
r['min_salary']=r['min_salary'].apply(lambda x: x*12 if x < 11000 else x)
r['max_salary']=r['max_salary'].apply(lambda x: x*12 if x < 11000 else x)


# Finally getting the average between min and max salary
r['Salary']=r[['min_salary','max_salary']].mean(axis=1)
r.reset_index(inplace=True)
r.drop(columns=['index'],inplace=True)

r.head()

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary
0,"Data ScientistTikTok3.5Mountain View, CA+27 lo...",TikTok,"Mountain View, CA+27 locations",NaN,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""...",NaN,NaN
1,newClinical Studies Data ScientistTailored Man...,Tailored Management,Remote,113027.2,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""...",113027.2,113027.2
2,newData Scientist or BioinformaticianUT Southw...,UT Southwestern Medical Center,+4 locationsRemote,80000.0,http://www.indeed.com/company/UT-Southwestern-...,"<div class=""jobsearch-jobDescriptionText"" id=""...",60000.0,100000.0
3,Data Scientist (Entry Level- Remote)HiLabsRale...,HiLabs,"Raleigh-Durham, NC+1 location•Remote",135000.0,http://www.indeed.com/company/HILABS/jobs/Data...,"<div class=""jobsearch-jobDescriptionText"" id=""...",120000.0,150000.0
4,newData Scientist - Full RemoteVaco3.7+1 locat...,Vaco,+1 locationRemote,157500.0,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""...",145000.0,170000.0


* Title

In [39]:
def title(tit):
    if 'data scientist' in tit.lower():
        return 'data scientist'
    elif 'data engineer' in tit.lower():
        return 'data engineer'
    elif 'analyst' in tit.lower():
        return 'analyst'
    elif 'machine learning' in tit.lower():
        return 'Mle'
    elif 'manager' in tit.lower():
        return 'manager'
    elif 'lead' in tit.lower():
        return 'lead'
    elif 'director' in tit.lower():
        return 'director'
    elif 'ml' in tit.lower():
        return 'Mle'
    elif 'artificial intelligence' in tit.lower():
        return 'AI'
    else:
        return ''
    
r['Title']=r['Title'].apply(title)
r=r.replace(r'^\s*$', np.NaN, regex=True)
r.Title.value_counts()

data scientist    703
data engineer     105
analyst            34
Mle                21
AI                  3
manager             2
Name: Title, dtype: int64

* Location

In [40]:
# adding The Remote column and it has a boolean type
r['Remote']=r['Location'].apply(lambda x: True if 'Remote' in x else False)

r['Location']=r['Location'].apply(lambda x: x.split('+')[0])
r['Location']=r['Location'].apply(lambda x: x.split('•')[0])
r['Location']=r['Location'].apply(lambda x: x.split('(')[0])
r['Location']=r['Location'].apply(lambda x: x.strip())
r['city']=r['Location'].apply(lambda x: x.split(',')[0])
r['state']=r['Location'].apply(lambda x: x.split(',')[1] if ',' in x else x)

r['city']=r['city'].astype(str)
r['city']=r['city'].apply(lambda x: '' if 'Remote' in x else x)

r['state']=r['state'].astype(str)
r['state']=r['state'].apply(lambda x: '' if 'Remote' in x else x)
df['Location']=df['Location'].apply(lambda x: np.NaN if x=='Remote' else x)
r=r.replace(r'^\s*$', np.NaN, regex=True)

* state and city

In [41]:
# replacing state with their full names into their abbreviation names
r['state']=r['state'].astype(str)
r['state']=r['state'].apply(lambda x: x.strip())
r['state']=r['state'].apply(lambda x: x.replace('New York State','NY'))
r['state']=r['state'].apply(lambda x: x.replace('Washington State','WA'))
r['state']=r['state'].apply(lambda x: x.replace('United States',''))
r['state']=r['state'].apply(lambda x: x.replace('Texas','TX'))
r['state']=r['state'].apply(lambda x: x.replace('Virginia','VA'))
r['state']=r['state'].apply(lambda x: x.replace('Massachusetts','MA'))
r['state']=r['state'].apply(lambda x: x.replace('California','CA'))
r['state']=r['state'].apply(lambda x: x.replace('Colorado','CO'))
r['state']=r['state'].apply(lambda x: x.replace('Florida','FL'))
r['state']=r['state'].apply(lambda x: x.replace('Georgia','GA'))
r['state']=r['state'].apply(lambda x: x.replace('Illinois','IL'))
r['state']=r['state'].apply(lambda x: x.replace('Iowa','IA'))
r['state']=r['state'].apply(lambda x: x.replace('Maryland','MD'))

r=r.replace(r'^\s*$', np.NaN, regex=True)
r['city']=r['city'].astype(str)
r['city']=r['city'].apply(lambda x: x.replace('New York State','New York'))
r['city']=r['city'].apply(lambda x: x.replace('San Francisco Bay Area','San Francisco'))
r['city']=r['city'].apply(lambda x: x.replace('California','San Francisco'))


r.head()

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,city,state
0,data scientist,TikTok,"Mountain View, CA",NaN,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""...",NaN,NaN,False,Mountain View,CA
1,data scientist,Tailored Management,Remote,113027.2,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""...",113027.2,113027.2,True,nan,nan
2,data scientist,UT Southwestern Medical Center,NaN,80000.0,http://www.indeed.com/company/UT-Southwestern-...,"<div class=""jobsearch-jobDescriptionText"" id=""...",60000.0,100000.0,True,nan,nan
3,data scientist,HiLabs,"Raleigh-Durham, NC",135000.0,http://www.indeed.com/company/HILABS/jobs/Data...,"<div class=""jobsearch-jobDescriptionText"" id=""...",120000.0,150000.0,True,Raleigh-Durham,NC
4,data scientist,Vaco,NaN,157500.0,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,"<div class=""jobsearch-jobDescriptionText"" id=""...",145000.0,170000.0,True,nan,nan


In [42]:
r['state']=r['state'].astype(str)
r['zip_code']=r['state'].apply(lambda x: x.split(' ')[1] if ' ' in x else '')
r['state']=r['state'].astype(str)
r['zip_code']=r['state'].apply(lambda x: x.split(' ')[1] if ' ' in x else '')
r['state']=r['state'].apply(lambda x: x.split(' ')[0] if ' ' in x else x)
r['city']=r['city'].astype(str)
r['city']=r['city'].apply(lambda x: x.replace('United States',''))
r=r.replace(r'^\s*$', np.NaN, regex=True)

r.tail()

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,city,state,zip_code
917,data engineer,NTERSOL,Remote,135000.0,http://www.indeed.com/company/NTERSOL/jobs/Dat...,"<div class=""jobsearch-jobDescriptionText"" id=""...",120000.0,150000.0,True,nan,nan,NaN
918,data engineer,Amazon.com Services LLC,"Santa Monica, CA 90404",NaN,http://www.indeed.com/rc/clk?jk=c4a261444f2522...,"<div class=""jobsearch-jobDescriptionText"" id=""...",NaN,NaN,False,Santa Monica,CA,90404
919,data engineer,ZLLIUS INC.,"Hoffman Estates, IL",75000.0,http://www.indeed.com/company/ZLLIUS-INC./jobs...,"<div class=""jobsearch-jobDescriptionText"" id=""...",65000.0,85000.0,True,Hoffman Estates,IL,NaN
920,data engineer,TECH AVEEV LLC,NaN,NaN,http://www.indeed.com/company/TECHAVEEV/jobs/D...,"<div class=""jobsearch-jobDescriptionText"" id=""...",NaN,NaN,True,nan,nan,NaN
921,NaN,Agama Solutions,"New York, NY 10017",147627.5,http://www.indeed.com/company/Agama-Solutions/...,"<div class=""jobsearch-jobDescriptionText"" id=""...",97291.0,197964.0,False,New York,NY,10017


* job_summary

In [43]:
# removing html tags with regex from job_summary
r['job_summary']=r['job_summary'].astype(str)
CLEANR = re.compile('<.*?>') 
def cleanhtml(raw_html):
    cleantext = re.sub(CLEANR, '', raw_html)
    return cleantext

r['job_summary']=r['job_summary'].apply(lambda x:cleanhtml(x))

# Finding the demanding skills for data scientists
s=['Python', 'SQL', 'AWS', 'RESTFUL', 'Machine learning', 'Deep Learning', 
        'Text Mining', 'NLP', 'SAS', 
        'Tableau', 'Sagemaker', 'TensorFlow', 'Spark','Power Bi','Git','Github',
        'Hadoop','Excel','Deep learning','Pandas','Statistics','Pythorch']

a=(map(lambda x: x.lower(),s))
skills=list(a)
skills

# converting job_summary to lower case in order to find skills easier
r['job_summary']=r['job_summary'].apply(lambda x: x.lower())

# creating new columns as skill names and returning True if that skill exists in job description
for i in skills:
    r[i]=r['job_summary'].apply(lambda x: True if i in x else False)


# finding skills for the R programming language
rs = []
x = r.job_summary
for i in x:
    split = set(i.split())
    x = set('R'.split()) & split
    if len(x) != 0:
        rs.append(True)
    else:
        rs.append(False)
r['R']=rs

r.head(15)

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,city,...,spark,power bi,git,github,hadoop,excel,pandas,statistics,pythorch,R
0,data scientist,TikTok,"Mountain View, CA",NaN,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,tiktok is the leading destination for short-fo...,NaN,NaN,False,Mountain View,...,False,False,False,False,False,False,False,True,False,False
1,data scientist,Tailored Management,Remote,113027.2,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,clinical studies data scientist location: remo...,113027.2,113027.2,True,nan,...,False,False,False,False,False,False,False,True,False,False
2,data scientist,UT Southwestern Medical Center,NaN,80000.0,http://www.indeed.com/company/UT-Southwestern-...,center information: the quantitative biomedica...,60000.0,100000.0,True,nan,...,False,False,False,False,False,False,False,True,False,False
3,data scientist,HiLabs,"Raleigh-Durham, NC",135000.0,http://www.indeed.com/company/HILABS/jobs/Data...,"at hilabs , our mission is to put healthcare o...",120000.0,150000.0,True,Raleigh-Durham,...,True,False,False,False,True,True,False,True,False,False
4,data scientist,Vaco,NaN,157500.0,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,skills and experience:\ntitle: data scientist\...,145000.0,170000.0,True,nan,...,True,False,True,False,False,False,True,False,False,False
5,data scientist,Starbucks,"Seattle, WA",NaN,http://www.indeed.com/rc/clk?jk=f84d4861a4473b...,"job posting\n: jan 13, 2022\njob posting end d...",NaN,NaN,True,Seattle,...,False,False,False,False,False,False,False,False,False,False
6,data scientist,Innover Global Inc.,"North Chicago, IL",145600.0,http://www.indeed.com/company/Innover-Global-I...,data scientistclassical data scientist prefera...,124800.0,166400.0,True,North Chicago,...,False,False,False,False,False,False,False,False,False,False
7,data scientist,Central Intelligence Agency,"Washington, DC 20505",121325.0,http://www.indeed.com/rc/clk?jk=a1c59666bc32cd...,data scientists develop algorithms and statist...,70150.0,172500.0,False,Washington,...,False,False,False,False,False,False,False,True,False,False
8,data scientist,Amazon.com Services LLC,"Seattle, WA",NaN,http://www.indeed.com/rc/clk?jk=0336a6b876ed3c...,\nexperienced in handling large data sets usin...,NaN,NaN,False,Seattle,...,False,False,False,False,False,True,False,True,False,False
9,data scientist,ViacomCBS,"New York, NY 10036",NaN,http://www.indeed.com/rc/clk?jk=c71b72157f64f8...,viacomcbs (nasdaq: viac; viaca) is a leading g...,NaN,NaN,True,New York,...,False,False,True,False,False,False,False,True,False,False


* Adding Degree

In [44]:
def degree(deg):
    if 'bachelor' in deg.lower():
        return 1
    elif 'Bachelor' in deg.lower():
        return 1
    elif 'bachelor\s/' in deg.lower():
        return 1
    elif 'Bachelor\s/' in deg.lower():
        return 1
    elif 'undergraduate' in deg.lower():
        return 1
    elif 'Undergraduate' in deg.lower():
        return 1
    elif '4 year degree' in deg.lower():
        return 1
    elif '4 Year Degree' in deg.lower():
        return 1
    elif '4-year degree' in deg.lower():
        return 1
    elif '4-Year Degree' in deg.lower():
        return 1
    
    elif 'master' in deg.lower():
        return 2
    elif 'Master' in deg.lower():
        return 2
    elif 'master\'s/' in deg.lower():
        return 2
    elif 'm.s.' in deg.lower():
        return 2
    elif 'M.S.' in deg.lower():
        return 2
    elif 'm.s' in deg.lower():
        return 2
    elif 'Master' in deg.lower():
        return 2
    elif 'M.S' in deg.lower():
        return 2
    elif 'ms/' in deg.lower():
        return 2
    elif 'MS/' in deg.lower():
        return 2
    elif 'm.s/' in deg.lower():
        return 2
    elif 'M.S/' in deg.lower():
        return 2
    elif 'm.s./' in deg.lower():
        return 2
    elif 'master/' in deg.lower():
        return 2
    elif 'Master/' in deg.lower():
        return 2
    elif 'graduate' in deg.lower():
        return 2
    elif 'Graduate' in deg.lower():
        return 2
    elif 'ms degree' in deg.lower():
        return 2
    elif 'MS Degree' in deg.lower():
        return 2
    elif 'graduate degree' in deg.lower():
        return 2
    elif 'Graduate degree' in deg.lower():
        return 2
    
    elif 'phd' in deg.lower():
        return 3
    elif 'PHD' in deg.lower():
        return 3
    elif 'ph.d' in deg.lower():
        return 3
    elif 'PH.D' in deg.lower():
        return 3
    elif 'phd/' in deg.lower():
        return 3
    elif 'PHD/' in deg.lower():
        return 3
    elif 'doctoral' in deg.lower():
        return 3
    elif 'Doctoral' in deg.lower():
        return 3
    elif 'doctorate' in deg.lower():
        return 3
    elif 'Doctorate' in deg.lower():
        return 3
    elif 'advanced degree' in deg.lower():
        return 3
    elif 'Advanced Degree' in deg.lower():
        return 3
    else:
        return ''
    
r['degree']=r['job_summary'].apply(lambda x: degree(x))
r=r.replace(r'^\s*$', np.NaN, regex=True)


r.head()

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,city,...,power bi,git,github,hadoop,excel,pandas,statistics,pythorch,R,degree
0,data scientist,TikTok,"Mountain View, CA",NaN,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,tiktok is the leading destination for short-fo...,NaN,NaN,False,Mountain View,...,False,False,False,False,False,False,True,False,False,2.0
1,data scientist,Tailored Management,Remote,113027.2,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,clinical studies data scientist location: remo...,113027.2,113027.2,True,nan,...,False,False,False,False,False,False,True,False,False,1.0
2,data scientist,UT Southwestern Medical Center,NaN,80000.0,http://www.indeed.com/company/UT-Southwestern-...,center information: the quantitative biomedica...,60000.0,100000.0,True,nan,...,False,False,False,False,False,False,True,False,False,3.0
3,data scientist,HiLabs,"Raleigh-Durham, NC",135000.0,http://www.indeed.com/company/HILABS/jobs/Data...,"at hilabs , our mission is to put healthcare o...",120000.0,150000.0,True,Raleigh-Durham,...,False,False,False,True,True,False,True,False,False,1.0
4,data scientist,Vaco,NaN,157500.0,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,skills and experience:\ntitle: data scientist\...,145000.0,170000.0,True,nan,...,False,True,False,False,False,True,False,False,False,1.0


In [45]:
def degr(deg):
    if deg==1.0:
        return 'Bachelor'
    elif deg==2.0:
        return "Master's"
    elif deg==3.0:
        return 'PHD'
    else:
        return ''

# Creating another column by just having the degree names instead of numbers
r['Degree']=r['degree'].apply(lambda x:degr(x))
r=r.replace(r'^\s*$', np.NaN, regex=True)

r.tail(10)

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,city,...,git,github,hadoop,excel,pandas,statistics,pythorch,R,degree,Degree
912,data engineer,"Baanyan Software Services, Inc",Remote,104000.0,http://www.indeed.com/company/Baanyan-Software...,job description: title: data engineer(kafka)lo...,83200.0,124800.0,True,nan,...,False,False,False,True,False,False,False,False,NaN,NaN
913,data engineer,NexTraq LLC,"Atlanta, GA 30319",NaN,http://www.indeed.com/rc/clk?jk=3a290e10f7e79c...,"the opportunity\nnextraq, a subsidiary of mich...",NaN,NaN,True,Atlanta,...,False,False,False,True,False,False,False,False,NaN,NaN
914,data engineer,Infinity Quest,"Thousand Oaks, CA",140400.0,http://www.indeed.com/company/Infinity-Quest/j...,requirements:6-8 years of experience with stro...,124800.0,156000.0,True,Thousand Oaks,...,False,False,False,False,False,False,False,False,2.0,Master's
915,NaN,SECU,"Wade, NC",NaN,http://www.indeed.com/rc/clk?jk=31d4f300968855...,if you are motivated and believe in the credit...,NaN,NaN,False,Wade,...,False,False,False,False,False,False,False,False,NaN,NaN
916,data engineer,AGCO,"Duluth, GA 30096",NaN,http://www.indeed.com/rc/clk?jk=c42143a81c7d54...,"agco, a global leader in the design, manufactu...",NaN,NaN,False,Duluth,...,False,False,True,False,False,False,False,False,1.0,Bachelor
917,data engineer,NTERSOL,Remote,135000.0,http://www.indeed.com/company/NTERSOL/jobs/Dat...,senior data engineerfull time and 100% remote!...,120000.0,150000.0,True,nan,...,False,False,False,False,False,False,False,False,NaN,NaN
918,data engineer,Amazon.com Services LLC,"Santa Monica, CA 90404",NaN,http://www.indeed.com/rc/clk?jk=c4a261444f2522...,"\nbachelor's degree in computer science, engin...",NaN,NaN,False,Santa Monica,...,False,False,True,False,False,True,False,False,1.0,Bachelor
919,data engineer,ZLLIUS INC.,"Hoffman Estates, IL",75000.0,http://www.indeed.com/company/ZLLIUS-INC./jobs...,"hi,greetings from zllius inc!we have an urgent...",65000.0,85000.0,True,Hoffman Estates,...,False,False,False,False,False,False,False,False,NaN,NaN
920,data engineer,TECH AVEEV LLC,NaN,NaN,http://www.indeed.com/company/TECHAVEEV/jobs/D...,job description:7+ years’ experience in data e...,NaN,NaN,True,nan,...,False,False,False,False,False,False,False,False,NaN,NaN
921,NaN,Agama Solutions,"New York, NY 10017",147627.5,http://www.indeed.com/company/Agama-Solutions/...,5+ years of devops experience including hands ...,97291.0,197964.0,False,New York,...,False,False,True,False,False,False,False,False,NaN,NaN


In [46]:
r.to_csv('Cleaned_indeed.pkl')

* Long and Lat being added

In [76]:
df=pd.read_csv('Cleaned_indeed.pkl')

In [89]:
df

,Unnamed: 0,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,...,git,github,hadoop,excel,pandas,statistics,pythorch,R,degree,Degree
0,0,data scientist,TikTok,"Mountain View, CA",NaN,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,tiktok is the leading destination for short-fo...,NaN,NaN,False,...,False,False,False,False,False,True,False,False,2.0,Master's
1,1,data scientist,Tailored Management,NaN,113027.2,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,clinical studies data scientist location: remo...,113027.2,113027.2,True,...,False,False,False,False,False,True,False,False,1.0,Bachelor
2,2,data scientist,UT Southwestern Medical Center,NaN,80000.0,http://www.indeed.com/company/UT-Southwestern-...,center information: the quantitative biomedica...,60000.0,100000.0,True,...,False,False,False,False,False,True,False,False,3.0,PHD
3,3,data scientist,HiLabs,"Raleigh-Durham, NC",135000.0,http://www.indeed.com/company/HILABS/jobs/Data...,"at hilabs , our mission is to put healthcare o...",120000.0,150000.0,True,...,False,False,True,True,False,True,False,False,1.0,Bachelor
4,4,data scientist,Vaco,NaN,157500.0,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,skills and experience:\ntitle: data scientist\...,145000.0,170000.0,True,...,True,False,False,False,True,False,False,False,1.0,Bachelor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917,917,data engineer,NTERSOL,NaN,135000.0,http://www.indeed.com/company/NTERSOL/jobs/Dat...,senior data engineerfull time and 100% remote!...,120000.0,150000.0,True,...,False,False,False,False,False,False,False,False,NaN,NaN
918,918,data engineer,Amazon.com Services LLC,"Santa Monica, CA 90404",NaN,http://www.indeed.com/rc/clk?jk=c4a261444f2522...,"\nbachelor's degree in computer science, engin...",NaN,NaN,False,...,False,False,True,False,False,True,False,False,1.0,Bachelor
919,919,data engineer,ZLLIUS INC.,"Hoffman Estates, IL",75000.0,http://www.indeed.com/company/ZLLIUS-INC./jobs...,"hi,greetings from zllius inc!we have an urgent...",65000.0,85000.0,True,...,False,False,False,False,False,False,False,False,NaN,NaN
920,920,data engineer,TECH AVEEV LLC,NaN,NaN,http://www.indeed.com/company/TECHAVEEV/jobs/D...,job description:7+ years’ experience in data e...,NaN,NaN,True,...,False,False,False,False,False,False,False,False,NaN,NaN


In [90]:
n=Nominatim(user_agent="my_app")
df['longitude']=df['city'].apply(lambda x: n.geocode(x).longitude)
df['latitude']=df['city'].apply(lambda x: n.geocode(x).latitude)

# dropping values which where NaN in cities but geopy counted them as an actual city
df['longitude']=df['longitude'].apply(lambda x: np.NaN if x ==11.0480288 else x)
df['latitude']=df['latitude'].apply(lambda x: np.NaN if x ==46.3144754 else x)





In [92]:
# Only rows with lat and long info
df=df[df['latitude'].notna()]

In [94]:
df.reset_index(inplace=True)

In [100]:
df

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,city,...,hadoop,excel,pandas,statistics,pythorch,R,degree,Degree,longitude,latitude
0,data scientist,TikTok,"Mountain View, CA",NaN,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,tiktok is the leading destination for short-fo...,NaN,NaN,False,Mountain View,...,False,False,False,True,False,False,2.0,Master's,-122.083210,37.389389
1,data scientist,HiLabs,"Raleigh-Durham, NC",135000.0,http://www.indeed.com/company/HILABS/jobs/Data...,"at hilabs , our mission is to put healthcare o...",120000.0,150000.0,True,Raleigh-Durham,...,True,True,False,True,False,False,1.0,Bachelor,-81.281286,37.689209
2,data scientist,Starbucks,"Seattle, WA",NaN,http://www.indeed.com/rc/clk?jk=f84d4861a4473b...,"job posting\n: jan 13, 2022\njob posting end d...",NaN,NaN,True,Seattle,...,False,False,False,False,False,False,NaN,NaN,-122.330062,47.603832
3,data scientist,Innover Global Inc.,"North Chicago, IL",145600.0,http://www.indeed.com/company/Innover-Global-I...,data scientistclassical data scientist prefera...,124800.0,166400.0,True,North Chicago,...,False,False,False,False,False,False,2.0,Master's,-87.841182,42.325578
4,data scientist,Central Intelligence Agency,"Washington, DC 20505",121325.0,http://www.indeed.com/rc/clk?jk=a1c59666bc32cd...,data scientists develop algorithms and statist...,70150.0,172500.0,False,Washington,...,False,False,False,True,False,False,1.0,Bachelor,-77.036543,38.895037
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,NaN,SECU,"Wade, NC",NaN,http://www.indeed.com/rc/clk?jk=31d4f300968855...,if you are motivated and believe in the credit...,NaN,NaN,False,Wade,...,False,False,False,False,False,False,NaN,NaN,-68.233338,46.799355
756,data engineer,AGCO,"Duluth, GA 30096",NaN,http://www.indeed.com/rc/clk?jk=c42143a81c7d54...,"agco, a global leader in the design, manufactu...",NaN,NaN,False,Duluth,...,True,False,False,False,False,False,1.0,Bachelor,-92.125122,46.772932
757,data engineer,Amazon.com Services LLC,"Santa Monica, CA 90404",NaN,http://www.indeed.com/rc/clk?jk=c4a261444f2522...,"\nbachelor's degree in computer science, engin...",NaN,NaN,False,Santa Monica,...,True,False,False,True,False,False,1.0,Bachelor,-118.491227,34.019470
758,data engineer,ZLLIUS INC.,"Hoffman Estates, IL",75000.0,http://www.indeed.com/company/ZLLIUS-INC./jobs...,"hi,greetings from zllius inc!we have an urgent...",65000.0,85000.0,True,Hoffman Estates,...,False,False,False,False,False,False,NaN,NaN,-88.079278,42.042726


In [142]:
# Geopy got a wrong long and lat from locations like London,OR or Moscow,ID
# So I had to hard code the correct long and lat for some of the locations!

df.at[313,'longitude']= -76.861046
df.at[313,'latitude']= 39.2037


df.at[432,'latitude']= 33.950001
df.at[432,'longitude']= -83.383331

df.at[233,'latitude']= 33.883991
df.at[233,'longitude']= -84.514374

df.at[165,'latitude']= 42.652580
df.at[165,'longitude']= -73.756233

df.at[148,'latitude']= 43.2249909 
df.at[148,'longitude']=  -75.783658 

df.at[207,'latitude']= 33.988270
df.at[207,'longitude']= -118.472023

df.at[171,'latitude']= 38.897980 
df.at[171,'longitude']=  -77.438683

df.at[451,'latitude']= 38.897980 
df.at[451,'longitude']=  -77.438683

df.at[503,'latitude']= 38.897980 
df.at[503,'longitude']=  -77.438683


df.at[434,'latitude']= 26.893070
df.at[434,'longitude']=   -80.058220

df.at[357,'latitude']= 26.244526
df.at[357,'longitude']= -80.206436


df.at[565,'latitude']= 43.071568
df.at[565,'longitude']= -70.762245

df.at[66,'latitude']= 37.8035378
df.at[66,'longitude']= -121.9794043

df.at[614,'latitude']= 26.893070
df.at[614,'longitude']=   -80.058220

df.at[609,'latitude']= 40.7920
df.at[609,'longitude']= -73.8096


df.at[407,'latitude']= 42.598713
df.at[407,'longitude']= -71.363625

df.at[230,'latitude']= 44.000000
df.at[230,'longitude']= -120.500000

df.at[568,'latitude']= 39.025665
df.at[568,'longitude']= -77.076367

df.at[34,'latitude']= 42.373611
df.at[34,'longitude']= -71.110558

df.at[102,'latitude']= 42.373611
df.at[102,'longitude']= -71.110558

df.at[126,'latitude']= 42.373611
df.at[126,'longitude']= -71.110558

df.at[130,'latitude']= 42.373611
df.at[130,'longitude']= -71.110558

df.at[174,'latitude']= 42.373611
df.at[174,'longitude']= -71.110558

df.at[218,'latitude']= 42.373611
df.at[218,'longitude']= -71.110558

df.at[231,'latitude']= 42.373611
df.at[231,'longitude']= -71.110558

df.at[237,'latitude']= 42.373611
df.at[237,'longitude']= -71.110558

df.at[329,'latitude']= 42.373611
df.at[329,'longitude']= -71.110558

df.at[332,'latitude']= 42.373611
df.at[332,'longitude']= -71.110558

df.at[360,'latitude']= 42.373611
df.at[360,'longitude']= -71.110558

df.at[566,'latitude']= 42.373611
df.at[566,'longitude']= -71.110558

df.at[569,'latitude']= 42.373611
df.at[569,'longitude']= -71.110558

df.at[633,'latitude']= 42.373611
df.at[633,'longitude']= -71.110558

df.at[670,'latitude']= 42.373611
df.at[670,'longitude']= -71.110558

df.at[523,'latitude']= 36.850769
df.at[523,'longitude']= -76.285873

df.at[108,'latitude']= 40.038582
df.at[108,'longitude']= -75.557594

df.at[120,'latitude']= 40.038582
df.at[120,'longitude']= -75.557594

df.at[211,'latitude']= 40.038582
df.at[211,'longitude']= -75.557594

df.at[399,'latitude']= 42.2959
df.at[399,'longitude']= -71.7128

df.at[326,'latitude']= 41.5084
df.at[326,'longitude']= -72.9106

df.at[330,'latitude']= 40.0992
df.at[330,'longitude']= -83.1141

df.at[721,'latitude']= 40.0992
df.at[721,'longitude']= -83.1141

df.at[637,'latitude']= 28.0836
df.at[637,'longitude']= -80.6081

df.at[32,'latitude']= 46.7324
df.at[32,'longitude']= -117.0002

df.at[63,'latitude']= 37.6808
df.at[63,'longitude']= -122.4000


df.at[239,'latitude']= 30.2855
df.at[239,'longitude']= -97.6654


df.head(15)

,Title,Company,Location,Salary,URL,job_summary,min_salary,max_salary,Remote,city,...,hadoop,excel,pandas,statistics,pythorch,R,degree,Degree,longitude,latitude
0,data scientist,TikTok,"Mountain View, CA",NaN,http://www.indeed.com/pagead/clk?mo=r&ad=-6NYl...,tiktok is the leading destination for short-fo...,NaN,NaN,False,Mountain View,...,False,False,False,True,False,False,2.0,Master's,-122.083210,37.389389
1,data scientist,HiLabs,"Raleigh-Durham, NC",135000.0,http://www.indeed.com/company/HILABS/jobs/Data...,"at hilabs , our mission is to put healthcare o...",120000.0,150000.0,True,Raleigh-Durham,...,True,True,False,True,False,False,1.0,Bachelor,-81.281286,37.689209
2,data scientist,Starbucks,"Seattle, WA",NaN,http://www.indeed.com/rc/clk?jk=f84d4861a4473b...,"job posting\n: jan 13, 2022\njob posting end d...",NaN,NaN,True,Seattle,...,False,False,False,False,False,False,NaN,NaN,-122.330062,47.603832
3,data scientist,Innover Global Inc.,"North Chicago, IL",145600.0,http://www.indeed.com/company/Innover-Global-I...,data scientistclassical data scientist prefera...,124800.0,166400.0,True,North Chicago,...,False,False,False,False,False,False,2.0,Master's,-87.841182,42.325578
4,data scientist,Central Intelligence Agency,"Washington, DC 20505",121325.0,http://www.indeed.com/rc/clk?jk=a1c59666bc32cd...,data scientists develop algorithms and statist...,70150.0,172500.0,False,Washington,...,False,False,False,True,False,False,1.0,Bachelor,-77.036543,38.895037
5,data scientist,Amazon.com Services LLC,"Seattle, WA",NaN,http://www.indeed.com/rc/clk?jk=0336a6b876ed3c...,\nexperienced in handling large data sets usin...,NaN,NaN,False,Seattle,...,False,True,False,True,False,False,2.0,Master's,-122.330062,47.603832
6,data scientist,ViacomCBS,"New York, NY 10036",NaN,http://www.indeed.com/rc/clk?jk=c71b72157f64f8...,viacomcbs (nasdaq: viac; viaca) is a leading g...,NaN,NaN,True,New York,...,False,False,False,True,False,False,2.0,Master's,-74.006015,40.712728
7,data scientist,Spotify,"New York, NY",NaN,http://www.indeed.com/rc/clk?jk=cf1dfad6664913...,"data, research &amp; insights\ndata science\nw...",NaN,NaN,True,New York,...,False,False,True,True,False,False,NaN,NaN,-74.006015,40.712728
8,data scientist,Scripps Health,"San Diego, CA 92121",NaN,http://www.indeed.com/rc/clk?jk=7f8b4c18048bda...,scripps health administrative services support...,NaN,NaN,False,San Diego,...,False,True,False,False,False,False,1.0,Bachelor,-117.162773,32.717420
9,data scientist,Red Dot Storage,"Houston, TX",NaN,http://www.indeed.com/rc/clk?jk=ae80aaf769599c...,red dot storage is looking for a driven data s...,NaN,NaN,True,Houston,...,False,False,False,False,False,False,NaN,NaN,-95.367697,29.758938


In [143]:
df.fillna('Not Specified',inplace=True)
df.to_pickle('pickle_w_LongLat_noNaN.pkl')

/Users/raminstad/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
